In [1]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Check your GPU name


True
NVIDIA GeForce GTX 1650 Ti


In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from huggingface_hub import snapshot_download

# Define model name and target directory
model_name = "google/pegasus-cnn_dailymail"
local_dir = r"C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_model"

# Download the model files completely
snapshot_download(repo_id=model_name, local_dir=local_dir, local_dir_use_symlinks=False)

print("Model downloaded successfully to", local_dir)


c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

Model downloaded successfully to C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_model


In [4]:
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# Define the local model directory
local_dir = r"C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_model"

# Load tokenizer and model from local directory
tokenizer = PegasusTokenizer.from_pretrained(local_dir)
model = PegasusForConditionalGeneration.from_pretrained(local_dir).to("cuda" if torch.cuda.is_available() else "cpu")

print("Model loaded successfully from local directory!")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_model and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully from local directory!


In [5]:
def summarize_text(text):
    max_tokens = 1056

    # Tokenize input text
    input_tokens = tokenizer.encode(text, return_tensors="pt", truncation=False)
    input_length = input_tokens.shape[1]
    char_length = len(text)

    print(f"\n🔹 Original Text Length: {char_length} characters, {input_length} tokens")

    # Target summary length (~25% of original)
    target_length = max(30, int(input_length * 0.25))  # Minimum length 30 tokens

    if input_length <= max_tokens:
        # ✅ Normal summarization for short texts
        print("✅ Text is within token limit, summarizing directly...")
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_tokens).to(model.device)
        summary_ids = model.generate(
            **inputs,
            max_length=target_length + 30,  # Allow some buffer to avoid cutting off sentences
            min_length=target_length // 2,
            length_penalty=2.0,
            num_beams=5,
            early_stopping=True  # Ensures sentence completion
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        print(f"🔹 Summary Length: {len(summary)} characters, {len(tokenizer.encode(summary))} tokens")
        return summary

    else:
        # ✅ Handle long text with chunking
        print("⚠️ Text exceeds token limit, performing chunked summarization...")
        chunk_size = max_tokens - 56  # Keep buffer for special tokens
        chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

        print(f"🔹 Total Chunks: {len(chunks)} (each ~{chunk_size} chars)")

        summaries = []
        for i, chunk in enumerate(chunks):
            print(f"\n📝 Processing Chunk {i+1}/{len(chunks)}...")
            inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=max_tokens).to(model.device)
            summary_ids = model.generate(
                **inputs,
                max_length=(target_length // len(chunks)) + 30,
                min_length=(target_length // (2 * len(chunks))),
                length_penalty=2.0,
                num_beams=5,
                early_stopping=True
            )
            chunk_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(chunk_summary)

            print(f"✅ Chunk {i+1} Summary Length: {len(chunk_summary)} chars, {len(tokenizer.encode(chunk_summary))} tokens")

        final_summary = " ".join(summaries)  # Merge summaries properly
        print(f"\n🔹 Final Summary Length: {len(final_summary)} characters, {len(tokenizer.encode(final_summary))} tokens")
        return final_summary


In [6]:
text = """This paper analyzes the time-consuming analysis of
each cascading network module (PNet module, RNet
module and ONet module) in MTCNN, and finds that the
time-consuming of PNet module is the highest (about
70%). According to the results of time-consuming
analysis, two improved methods are proposed, one is to
reduce the number of candidate face frames in input PNet
network and the other is to reduce the number of output
face frames in PNet network. Then, aiming at the
problem that MTCNN algorithm has low detection speed
in high-resolution video and cannot meet real-time
requirements, a series of optimization such as adjusting
the minnisize parameter and PNet threshold in
combination with the low computing power application
scenarios of the channel bayonet. It is verified in the
FDDB face test set and practical application, the
detection speed has increased by 70.1% when the
detection rate has dropped by only 3.5%, and the
improved scheme has achieved good results. Compared
with the performance of OpenCV-VJ and SURF face
detection algorithms on FDDB, the optimized MTCNN
algorithm has better performance. Through the analysis
of the detection results of the specific FDDB data set
pictures, it is found that the undetected face conditions do
not meet the actual application scenarios in this article,
which proves that the optimized algorithm has excellent
performance in actual applications. The test results reflect
that the reproduced and optimized MTCNN face
detection algorithm has good robustness to face pose
changes, and fully meets the requirements of face
recognition systems in low computing power scenarios
such as channel bayonet. A mature face recognition system usually consists of
image acquisition, image preprocessing, face detection,
face tracking, face alignment, feature extraction and
comparison. Among the more critical steps are face
detection, tracking and face feature extraction. In
recent years, face recognition systems have been
widely used in channel bayonet systems such as smart
access control and identity verification in high-speed
railway stations. These channel bayonet face
recognition systems have all or most of the face image
collection, face detection, face alignment, face quality
detection, face feature extraction, face tracking and
other steps. However, some of these systems require a
high degree of cooperation from people, some are
complex to implement, and some have high
requirements for hardware such as computing devices.
On the one hand, the computing power of embedded
systems is not enough to support face detection,
tracking and face feature pairing based on deep
learning. Real-time requirements, some channel
bayonet face recognition systems require people to
deliberately approach the camera to cooperate with the
system for verification, discarding the natural and
convenient advantages of face recognition. The
specific target scenario studied in this paper is a singlechannel bayonet (single face close range), and the goal
is to be able to quickly compare and recognize faces
within 1-4 meters. The goal of the research is to apply
a faster and better performance algorithm to the
channel bayonet face recognition system with low
computing power, and to improve the operating speed
of the face recognition system through the improved
face detection algorithm. It can be mounted on low-end
devices with poor computing performance while
maintaining certain detection and recognition performance.
Aiming at the above problems, this paper combines
the channel bayonet application scenarios, through
research and improve face detection algorithms, face
tracking auxiliary algorithms, and introduce face
selection algorithms in the face recognition module to
satisfy the real-time and convenience requirements of
the channel bayonet face recognition system. After investigating and studying the existing face
recognition system and analyzing its advantages and
disadvantages, this paper improves the key
technologies of the face recognition system. Firstly,
using Python language to reproduce the MTCNN face
detection algorithm on the TensorFlow deep learning
framework, and it is optimized and improved by
combining with the actual application scene of channel
bayonet in this paper. The fusion of the improved
speed-up MTCNN algorithm and the supervised
Kalman filter human tracking algorithm proposed in
this paper accelerates the system operation speed.
According to the face selection algorithm proposed in
this paper, a face selection module with multiple
purposes is designed. Then according to the
comparison and analysis of various face recognition
algorithms, the FaceNet face recognition algorithm is
selected, and the face recognition module is designed
in combination with the face selection algorithm
proposed in this paper. Finally, the above research is
applied to the channel bayonet face recognition system,
and the system design and implementation are
completed. The chapter arrangement and corresponding
main work of this paper are as follows.
The first part is the introduction and the second part
is overseas and domestic research status, which
explains the research background and significance of
face recognition system; This paper analyzes the
development status of face detection in the system at
home and abroad, and compares and analyzes various
algorithms, paving the way and explaining the reasons
for the selection of various algorithms in the following
research work. The third part introduces the common
data set of face detection. The fourth part and the fifth
part choose MTCNN algorithm as the face detection
algorithm in the system according to the investigation
and experimental comparison of face detection algorithm.
Since there is no engineering implementation of
MTCNN algorithm based on Python language and
TensorFlow framework, it is reproduced and trained
after an in-depth understanding of its convolutional
neural network structure and detailed training methods.
Afterwards, the time-consuming analysis of all levels
of the network in MTCNN was carried out, and it was
found that the PNet module took the highest proportion
of time, and the corresponding speed-up suggestions
were put forward. Then, in view of the low detection
speed of the MTCNN algorithm in high-resolution
videos, it cannot meet the real-time problem, combined
with the specific application scenarios of the channel
bayonet face recognition in this paper, a series of
optimizations such as minnisize adjustment and PNet
threshold adjustment are performed. Finally, according
to the detection rate on the FDDB face test set and the
actual detection speed in 720p high-resolution video, it
can be seen that the detection speed increases by
70.1% when the detection rate drops by 3.5%. By
analyzing the detection results of specific FDDB data
set images, it is found that the undetected face
conditions do not meet the practical application
scenarios in this paper, which proves the excellent
performance of the optimized algorithm in practical
application. Finally, the face detection module was
designed and implemented, and actual engineering
tests were carried out; the test results reflect the
specific data that the reproduced and tuned MTCNN
face detection algorithm has good robustness to face
pose changes and fully meet the requirements of the
channel bayonet face recognition system. The sixth
part is a summary and outlook. It summarizes the main
research content of this paper, analyzes the affirmative
part of the research work and the problems that need to
be studied, and put forward the research direction for
future work. """  # Replace with real content
summary = summarize_text(text)

print("\n✨ FINAL SUMMARY:\n", summary)


Token indices sequence length is longer than the specified maximum sequence length for this model (1337 > 1024). Running this sequence through the model will result in indexing errors



🔹 Original Text Length: 7626 characters, 1337 tokens
⚠️ Text exceeds token limit, performing chunked summarization...
🔹 Total Chunks: 8 (each ~1000 chars)

📝 Processing Chunk 1/8...
✅ Chunk 1 Summary Length: 322 chars, 70 tokens

📝 Processing Chunk 2/8...
✅ Chunk 2 Summary Length: 238 chars, 52 tokens

📝 Processing Chunk 3/8...
✅ Chunk 3 Summary Length: 260 chars, 49 tokens

📝 Processing Chunk 4/8...
✅ Chunk 4 Summary Length: 225 chars, 44 tokens

📝 Processing Chunk 5/8...
✅ Chunk 5 Summary Length: 272 chars, 50 tokens

📝 Processing Chunk 6/8...
✅ Chunk 6 Summary Length: 278 chars, 51 tokens

📝 Processing Chunk 7/8...
✅ Chunk 7 Summary Length: 327 chars, 70 tokens

📝 Processing Chunk 8/8...
✅ Chunk 8 Summary Length: 248 chars, 45 tokens

🔹 Final Summary Length: 2177 characters, 424 tokens

✨ FINAL SUMMARY:
 This paper analyzes the time-consuming analysis of each cascading network module (PNet module, RNet module and ONet module) in MTCNN .<n>Two improved methods are proposed, one is t

In [1]:
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# Define the local model directory
local_dir = r"C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_model"

# Load tokenizer and model from local directory
tokenizer = PegasusTokenizer.from_pretrained(local_dir)
model = PegasusForConditionalGeneration.from_pretrained(local_dir).to("cuda" if torch.cuda.is_available() else "cpu")

print("Model loaded successfully from local directory!")





Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at C:\Users\nihar\Desktop\8th Sem\Hugging_Face\pegasus_model and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully from local directory!


In [2]:
def chunk_text(text, tokenizer, max_tokens=1024):
    """
    Splits long text into token-based chunks while maintaining sentence boundaries.
    """
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    
    print(f"🔹 Total Input Tokens: {len(tokens)}")
    print(f"🔹 Number of Chunks: {len(chunks)}")
    print(f"🔹 Chunk Sizes: {[len(chunk) for chunk in chunks]}\n")
    
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks], len(tokens)

def summarize_text(text, min_ratio=0.25, max_ratio=0.30):
    """
    Summarizes the text ensuring that the summary length is 25-30% of the original.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Token-based chunking for long texts
    max_input_tokens = 1024  # Pegasus supports 1024 tokens
    text_chunks, total_input_tokens = chunk_text(text, tokenizer, max_tokens=max_input_tokens)

    summaries = []
    for i, chunk in enumerate(text_chunks):
        input_tokens = tokenizer.encode(chunk, return_tensors="pt", truncation=True).to(device)
        input_length = input_tokens.shape[1]
        
        # Ensure 25-30% summary length
        min_summary_length = max(20, int(input_length * min_ratio))
        max_summary_length = max(min_summary_length + 15, int(input_length * max_ratio))  

        print(f"🟢 Chunk {i+1}:")
        print(f"   🔸 Input Tokens: {input_length}")
        print(f"   🔸 Expected Output Tokens: {min_summary_length} - {max_summary_length}")

        # Generate summary
        summary_ids = model.generate(
            input_tokens, 
            max_length=max_summary_length, 
            min_length=min_summary_length, 
            length_penalty=1.5, 
            num_return_sequences=1
        )

        output_tokens = summary_ids.shape[1]
        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        print(f"   🔹 Output Tokens: {output_tokens}")
        print(f"   🔹 Summary: {summary_text}\n")

        summaries.append(summary_text)

    # Re-summarize the merged summaries if multiple chunks exist
    final_summary = " ".join(summaries)
    total_output_tokens = len(tokenizer.encode(final_summary, truncation=False))

    print(f"\n✅ Final Summary Tokens: {total_output_tokens} (Target: {int(total_input_tokens * min_ratio)} - {int(total_input_tokens * max_ratio)})")
    print(f"📌 Final Summary:\n{final_summary}\n")

    return final_summary.replace("\n", " ")  # Remove unwanted newlines

# Example Usage
# input_text = """Your long input text goes here... It can be multiple paragraphs long."""
# summary = summarize_text(input_text)

In [3]:
# Example Usage
input_text = """This paper analyzes the time-consuming analysis of
each cascading network module (PNet module, RNet
module and ONet module) in MTCNN, and finds that the
time-consuming of PNet module is the highest (about
70%). According to the results of time-consuming
analysis, two improved methods are proposed, one is to
reduce the number of candidate face frames in input PNet
network and the other is to reduce the number of output
face frames in PNet network. Then, aiming at the
problem that MTCNN algorithm has low detection speed
in high-resolution video and cannot meet real-time
requirements, a series of optimization such as adjusting
the minnisize parameter and PNet threshold in
combination with the low computing power application
scenarios of the channel bayonet. It is verified in the
FDDB face test set and practical application, the
detection speed has increased by 70.1% when the
detection rate has dropped by only 3.5%, and the
improved scheme has achieved good results. Compared
with the performance of OpenCV-VJ and SURF face
detection algorithms on FDDB, the optimized MTCNN
algorithm has better performance. Through the analysis
of the detection results of the specific FDDB data set
pictures, it is found that the undetected face conditions do
not meet the actual application scenarios in this article,
which proves that the optimized algorithm has excellent
performance in actual applications. The test results reflect
that the reproduced and optimized MTCNN face
detection algorithm has good robustness to face pose
changes, and fully meets the requirements of face
recognition systems in low computing power scenarios
such as channel bayonet. A mature face recognition system usually consists of
image acquisition, image preprocessing, face detection,
face tracking, face alignment, feature extraction and
comparison. Among the more critical steps are face
detection, tracking and face feature extraction. In
recent years, face recognition systems have been
widely used in channel bayonet systems such as smart
access control and identity verification in high-speed
railway stations. These channel bayonet face
recognition systems have all or most of the face image
collection, face detection, face alignment, face quality
detection, face feature extraction, face tracking and
other steps. However, some of these systems require a
high degree of cooperation from people, some are
complex to implement, and some have high
requirements for hardware such as computing devices.
On the one hand, the computing power of embedded
systems is not enough to support face detection,
tracking and face feature pairing based on deep
learning. Real-time requirements, some channel
bayonet face recognition systems require people to
deliberately approach the camera to cooperate with the
system for verification, discarding the natural and
convenient advantages of face recognition. The
specific target scenario studied in this paper is a singlechannel bayonet (single face close range), and the goal
is to be able to quickly compare and recognize faces
within 1-4 meters. The goal of the research is to apply
a faster and better performance algorithm to the
channel bayonet face recognition system with low
computing power, and to improve the operating speed
of the face recognition system through the improved
face detection algorithm. It can be mounted on low-end
devices with poor computing performance while
maintaining certain detection and recognition performance.
Aiming at the above problems, this paper combines
the channel bayonet application scenarios, through
research and improve face detection algorithms, face
tracking auxiliary algorithms, and introduce face
selection algorithms in the face recognition module to
satisfy the real-time and convenience requirements of
the channel bayonet face recognition system. After investigating and studying the existing face
recognition system and analyzing its advantages and
disadvantages, this paper improves the key
technologies of the face recognition system. Firstly,
using Python language to reproduce the MTCNN face
detection algorithm on the TensorFlow deep learning
framework, and it is optimized and improved by
combining with the actual application scene of channel
bayonet in this paper. The fusion of the improved
speed-up MTCNN algorithm and the supervised
Kalman filter human tracking algorithm proposed in
this paper accelerates the system operation speed.
According to the face selection algorithm proposed in
this paper, a face selection module with multiple
purposes is designed. Then according to the
comparison and analysis of various face recognition
algorithms, the FaceNet face recognition algorithm is
selected, and the face recognition module is designed
in combination with the face selection algorithm
proposed in this paper. Finally, the above research is
applied to the channel bayonet face recognition system,
and the system design and implementation are
completed. The chapter arrangement and corresponding
main work of this paper are as follows.
The first part is the introduction and the second part
is overseas and domestic research status, which
explains the research background and significance of
face recognition system; This paper analyzes the
development status of face detection in the system at
home and abroad, and compares and analyzes various
algorithms, paving the way and explaining the reasons
for the selection of various algorithms in the following
research work. The third part introduces the common
data set of face detection. The fourth part and the fifth
part choose MTCNN algorithm as the face detection
algorithm in the system according to the investigation
and experimental comparison of face detection algorithm.
Since there is no engineering implementation of
MTCNN algorithm based on Python language and
TensorFlow framework, it is reproduced and trained
after an in-depth understanding of its convolutional
neural network structure and detailed training methods.
Afterwards, the time-consuming analysis of all levels
of the network in MTCNN was carried out, and it was
found that the PNet module took the highest proportion
of time, and the corresponding speed-up suggestions
were put forward. Then, in view of the low detection
speed of the MTCNN algorithm in high-resolution
videos, it cannot meet the real-time problem, combined
with the specific application scenarios of the channel
bayonet face recognition in this paper, a series of
optimizations such as minnisize adjustment and PNet
threshold adjustment are performed. Finally, according
to the detection rate on the FDDB face test set and the
actual detection speed in 720p high-resolution video, it
can be seen that the detection speed increases by
70.1% when the detection rate drops by 3.5%. By
analyzing the detection results of specific FDDB data
set images, it is found that the undetected face
conditions do not meet the practical application
scenarios in this paper, which proves the excellent
performance of the optimized algorithm in practical
application. Finally, the face detection module was
designed and implemented, and actual engineering
tests were carried out; the test results reflect the
specific data that the reproduced and tuned MTCNN
face detection algorithm has good robustness to face
pose changes and fully meet the requirements of the
channel bayonet face recognition system. The sixth
part is a summary and outlook. It summarizes the main
research content of this paper, analyzes the affirmative
part of the research work and the problems that need to
be studied, and put forward the research direction for
future work. """
summary = summarize_text(input_text)
print("Final Summary:\n", summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (1337 > 1024). Running this sequence through the model will result in indexing errors


🔹 Total Input Tokens: 1337
🔹 Number of Chunks: 2
🔹 Chunk Sizes: [1024, 313]

🟢 Chunk 1:
   🔸 Input Tokens: 1024
   🔸 Expected Output Tokens: 256 - 307
   🔹 Output Tokens: 307
   🔹 Summary: This paper analyzes the time-consuming analysis of each network module (PNet module, RNet module and ONet module) in MTCNN.<n>It finds that the time-consuming of PNet module is the highest (about 70%)<n>Two improved methods are proposed, one is to reduce the number of candidate face frames in input PNet network and the other is to reduce the number of output face frames in PNet network.<n>It is verified in the FDDB face test set and practical application, the detection speed has increased by 70.1% when the detection rate has dropped by only 3.5%, and the improved scheme has achieved good results.<n>This paper combines the channel bayonet application scenarios, through research and improve face detection algorithms, face tracking auxiliary algorithms, and introduce face selection algorithms in the fac

In [6]:
x = input_text.split(" ")
print(len(x))
y = summary.split(" ")
print(len(y))

1011
321


In [4]:
a = summary.split("<n>")
print(len(a))
for i in a:
    print(i)

8
This paper analyzes the time-consuming analysis of each network module (PNet module, RNet module and ONet module) in MTCNN.
It finds that the time-consuming of PNet module is the highest (about 70%)
Two improved methods are proposed, one is to reduce the number of candidate face frames in input PNet network and the other is to reduce the number of output face frames in PNet network.
It is verified in the FDDB face test set and practical application, the detection speed has increased by 70.1% when the detection rate has dropped by only 3.5%, and the improved scheme has achieved good results.
This paper combines the channel bayonet application scenarios, through research and improve face detection algorithms, face tracking auxiliary algorithms, and introduce face selection algorithms in the face recognition module to satisfy the real-time and convenience requirements of the channel bayonet face recognition system.
The improved face detection algorithm can be mounted on low-end devices 

In [7]:
import torch
import nltk
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# Ensure nltk sentence tokenizer is available
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nihar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
def chunk_text(text, tokenizer, max_tokens=1024):
    """
    Splits long text into token-based chunks while maintaining sentence boundaries.
    """
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    
    print(f"🔹 Total Input Tokens: {len(tokens)}")
    print(f"🔹 Number of Chunks: {len(chunks)}")
    print(f"🔹 Chunk Sizes: {[len(chunk) for chunk in chunks]}\n")
    
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks], len(tokens)

def ensure_complete_sentences(text):
    """
    Trims the text to ensure it ends at a proper sentence boundary.
    """
    sentences = nltk.sent_tokenize(text)
    if not sentences:
        return text  # If no sentences are detected, return as is
    return " ".join(sentences[:-1]) if len(sentences) > 1 else sentences[0]  # Remove incomplete last sentence

def summarize_text(text, min_ratio=0.25, max_ratio=0.30):
    """
    Summarizes the text ensuring that the summary length is 25-30% of the original.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Token-based chunking for long texts
    max_input_tokens = 1024  # Pegasus supports 1024 tokens
    text_chunks, total_input_tokens = chunk_text(text, tokenizer, max_tokens=max_input_tokens)

    summaries = []
    for i, chunk in enumerate(text_chunks):
        input_tokens = tokenizer.encode(chunk, return_tensors="pt", truncation=True).to(device)
        input_length = input_tokens.shape[1]
        
        # Ensure 25-30% summary length with a buffer
        min_summary_length = max(20, int(input_length * min_ratio))
        max_summary_length = max(min_summary_length + 30, int(input_length * max_ratio))  # Added buffer of +30 tokens

        print(f"🟢 Chunk {i+1}:")
        print(f"   🔸 Input Tokens: {input_length}")
        print(f"   🔸 Expected Output Tokens: {min_summary_length} - {max_summary_length}")

        # Generate summary
        summary_ids = model.generate(
            input_tokens, 
            max_length=max_summary_length, 
            min_length=min_summary_length, 
            length_penalty=1.5, 
            num_return_sequences=1
        )

        output_tokens = summary_ids.shape[1]
        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        # Ensure sentences are complete
        summary_text = ensure_complete_sentences(summary_text)

        print(f"   🔹 Output Tokens: {output_tokens}")
        print(f"   🔹 Summary: {summary_text}\n")

        summaries.append(summary_text)

    # Re-summarize the merged summaries if multiple chunks exist
    final_summary = " ".join(summaries)
    total_output_tokens = len(tokenizer.encode(final_summary, truncation=False))

    print(f"\n✅ Final Summary Tokens: {total_output_tokens} (Target: {int(total_input_tokens * min_ratio)} - {int(total_input_tokens * max_ratio)})")
    print(f"📌 Final Summary:\n{final_summary}\n")

    return final_summary.replace("\n", " ")  # Remove unwanted newlines

In [12]:
# Example Usage
input_text = """This paper analyzes the time-consuming analysis of
each cascading network module (PNet module, RNet
module and ONet module) in MTCNN, and finds that the
time-consuming of PNet module is the highest (about
70%). According to the results of time-consuming
analysis, two improved methods are proposed, one is to
reduce the number of candidate face frames in input PNet
network and the other is to reduce the number of output
face frames in PNet network. Then, aiming at the
problem that MTCNN algorithm has low detection speed
in high-resolution video and cannot meet real-time
requirements, a series of optimization such as adjusting
the minnisize parameter and PNet threshold in
combination with the low computing power application
scenarios of the channel bayonet. It is verified in the
FDDB face test set and practical application, the
detection speed has increased by 70.1% when the
detection rate has dropped by only 3.5%, and the
improved scheme has achieved good results. Compared
with the performance of OpenCV-VJ and SURF face
detection algorithms on FDDB, the optimized MTCNN
algorithm has better performance. Through the analysis
of the detection results of the specific FDDB data set
pictures, it is found that the undetected face conditions do
not meet the actual application scenarios in this article,
which proves that the optimized algorithm has excellent
performance in actual applications. The test results reflect
that the reproduced and optimized MTCNN face
detection algorithm has good robustness to face pose
changes, and fully meets the requirements of face
recognition systems in low computing power scenarios
such as channel bayonet. A mature face recognition system usually consists of
image acquisition, image preprocessing, face detection,
face tracking, face alignment, feature extraction and
comparison. Among the more critical steps are face
detection, tracking and face feature extraction. In
recent years, face recognition systems have been
widely used in channel bayonet systems such as smart
access control and identity verification in high-speed
railway stations. These channel bayonet face
recognition systems have all or most of the face image
collection, face detection, face alignment, face quality
detection, face feature extraction, face tracking and
other steps. However, some of these systems require a
high degree of cooperation from people, some are
complex to implement, and some have high
requirements for hardware such as computing devices.
On the one hand, the computing power of embedded
systems is not enough to support face detection,
tracking and face feature pairing based on deep
learning. Real-time requirements, some channel
bayonet face recognition systems require people to
deliberately approach the camera to cooperate with the
system for verification, discarding the natural and
convenient advantages of face recognition. The
specific target scenario studied in this paper is a singlechannel bayonet (single face close range), and the goal
is to be able to quickly compare and recognize faces
within 1-4 meters. The goal of the research is to apply
a faster and better performance algorithm to the
channel bayonet face recognition system with low
computing power, and to improve the operating speed
of the face recognition system through the improved
face detection algorithm. It can be mounted on low-end
devices with poor computing performance while
maintaining certain detection and recognition performance.
Aiming at the above problems, this paper combines
the channel bayonet application scenarios, through
research and improve face detection algorithms, face
tracking auxiliary algorithms, and introduce face
selection algorithms in the face recognition module to
satisfy the real-time and convenience requirements of
the channel bayonet face recognition system. After investigating and studying the existing face
recognition system and analyzing its advantages and
disadvantages, this paper improves the key
technologies of the face recognition system. Firstly,
using Python language to reproduce the MTCNN face
detection algorithm on the TensorFlow deep learning
framework, and it is optimized and improved by
combining with the actual application scene of channel
bayonet in this paper. The fusion of the improved
speed-up MTCNN algorithm and the supervised
Kalman filter human tracking algorithm proposed in
this paper accelerates the system operation speed.
According to the face selection algorithm proposed in
this paper, a face selection module with multiple
purposes is designed. Then according to the
comparison and analysis of various face recognition
algorithms, the FaceNet face recognition algorithm is
selected, and the face recognition module is designed
in combination with the face selection algorithm
proposed in this paper. Finally, the above research is
applied to the channel bayonet face recognition system,
and the system design and implementation are
completed. The chapter arrangement and corresponding
main work of this paper are as follows.
The first part is the introduction and the second part
is overseas and domestic research status, which
explains the research background and significance of
face recognition system; This paper analyzes the
development status of face detection in the system at
home and abroad, and compares and analyzes various
algorithms, paving the way and explaining the reasons
for the selection of various algorithms in the following
research work. The third part introduces the common
data set of face detection. The fourth part and the fifth
part choose MTCNN algorithm as the face detection
algorithm in the system according to the investigation
and experimental comparison of face detection algorithm.
Since there is no engineering implementation of
MTCNN algorithm based on Python language and
TensorFlow framework, it is reproduced and trained
after an in-depth understanding of its convolutional
neural network structure and detailed training methods.
Afterwards, the time-consuming analysis of all levels
of the network in MTCNN was carried out, and it was
found that the PNet module took the highest proportion
of time, and the corresponding speed-up suggestions
were put forward. Then, in view of the low detection
speed of the MTCNN algorithm in high-resolution
videos, it cannot meet the real-time problem, combined
with the specific application scenarios of the channel
bayonet face recognition in this paper, a series of
optimizations such as minnisize adjustment and PNet
threshold adjustment are performed. Finally, according
to the detection rate on the FDDB face test set and the
actual detection speed in 720p high-resolution video, it
can be seen that the detection speed increases by
70.1% when the detection rate drops by 3.5%. By
analyzing the detection results of specific FDDB data
set images, it is found that the undetected face
conditions do not meet the practical application
scenarios in this paper, which proves the excellent
performance of the optimized algorithm in practical
application. Finally, the face detection module was
designed and implemented, and actual engineering
tests were carried out; the test results reflect the
specific data that the reproduced and tuned MTCNN
face detection algorithm has good robustness to face
pose changes and fully meet the requirements of the
channel bayonet face recognition system. The sixth
part is a summary and outlook. It summarizes the main
research content of this paper, analyzes the affirmative
part of the research work and the problems that need to
be studied, and put forward the research direction for
future work. """
summary = summarize_text(input_text)
print("Final Summary:\n", summary)

🔹 Total Input Tokens: 1337
🔹 Number of Chunks: 2
🔹 Chunk Sizes: [1024, 313]

🟢 Chunk 1:
   🔸 Input Tokens: 1024
   🔸 Expected Output Tokens: 256 - 307
   🔹 Output Tokens: 307
   🔹 Summary: This paper analyzes the time-consuming analysis of each network module (PNet module, RNet module and ONet module) in MTCNN.<n>It finds that the time-consuming of PNet module is the highest (about 70%)<n>Two improved methods are proposed, one is to reduce the number of candidate face frames in input PNet network and the other is to reduce the number of output face frames in PNet network.<n>It is verified in the FDDB face test set and practical application, the detection speed has increased by 70.1% when the detection rate has dropped by only 3.5%, and the improved scheme has achieved good results.<n>This paper combines the channel bayonet application scenarios, through research and improve face detection algorithms, face tracking auxiliary algorithms, and introduce face selection algorithms in the fac

In [13]:
a = summary.split("<n>")
print(len(a))
for i in a:
    print(i)

8
This paper analyzes the time-consuming analysis of each network module (PNet module, RNet module and ONet module) in MTCNN.
It finds that the time-consuming of PNet module is the highest (about 70%)
Two improved methods are proposed, one is to reduce the number of candidate face frames in input PNet network and the other is to reduce the number of output face frames in PNet network.
It is verified in the FDDB face test set and practical application, the detection speed has increased by 70.1% when the detection rate has dropped by only 3.5%, and the improved scheme has achieved good results.
This paper combines the channel bayonet application scenarios, through research and improve face detection algorithms, face tracking auxiliary algorithms, and introduce face selection algorithms in the face recognition module to satisfy the real-time and convenience requirements of the channel bayonet face recognition system.
The improved face detection algorithm can be mounted on low-end devices 

In [15]:
s = summary.replace("<n>"," ")
print(s)

This paper analyzes the time-consuming analysis of each network module (PNet module, RNet module and ONet module) in MTCNN. It finds that the time-consuming of PNet module is the highest (about 70%) Two improved methods are proposed, one is to reduce the number of candidate face frames in input PNet network and the other is to reduce the number of output face frames in PNet network. It is verified in the FDDB face test set and practical application, the detection speed has increased by 70.1% when the detection rate has dropped by only 3.5%, and the improved scheme has achieved good results. This paper combines the channel bayonet application scenarios, through research and improve face detection algorithms, face tracking auxiliary algorithms, and introduce face selection algorithms in the face recognition module to satisfy the real-time and convenience requirements of the channel bayonet face recognition system. The improved face detection algorithm can be mounted on low-end devices wi

In [16]:
x = input_text.split(" ")
print(len(x))
y = summary.split(" ")
print(len(y))

1011
329
